In [37]:
import random

# Define card values (dictionary for easy reference)
card_values = {
    "2": 2,
    "3": 3,
    "4": 4,
    "5": 5,
    "6": 6,
    "7": 7,
    "8": 8,
    "9": 9,
    "10": 10,
    "jack": 11,
    "queen": 12,
    "king": 13,
    "ace": 14
}

In [19]:
class Card:
  def __init__(self, suit, value):
    self.suit = suit
    self.value = value

  def get_value(self):
    return card_values[self.value]


In [20]:
class Player:
  def __init__(self, name):
    self.name = name
    self.deck = []  # Add deck attribute to store player's unique cards
    self.points = 0

  def draw_card(self):
    if len(self.deck) > 0:
        self.deck.pop()  # Draw from player's deck
    else:
      print(f"{self.name}'s deck is empty!")

  def bid(self):
    # Implement bidding strategy here (e.g., random choice, AI logic)
    chosen_card = random.choice(self.deck)
    #print(f"{self.name} bids: {chosen_card.value}")
    return chosen_card

  def update_points(self, round_points):
    self.points += round_points


In [21]:
class Banker:
  def __init__(self):
    self.diamond_deck = [Card("Diamonds", value) for value in card_values.keys()]
    random.shuffle(self.diamond_deck)  # Shuffle diamonds before dealing

  def generate_diamond_card(self):
    if len(self.diamond_deck) > 0:
      revealed_diamond = self.diamond_deck.pop()  # Draw and remove a diamond
      return revealed_diamond
    else:
      # Handle situation where all diamonds are used (game ends after this round)
      return None


In [22]:
def calculate_diamond_probability( remaining_diamonds):
  """
  This function calculates the probability of each diamond value being revealed.

  Args:
      revealed_diamonds: List of revealed diamond values (integers).

  Returns:
      Dictionary: Probabilities for each diamond value (2-14).
  """

  diamond_probs_of_remaining_cards = { card : remaining_diamonds.count(card) / (len(remaining_diamonds))\
                                      for card in remaining_diamonds }
  return diamond_probs_of_remaining_cards

from functools import reduce

def probability_of_winning(individual_player_cards , card_to_win):
  """
  This function calculates the probability of a card winning the trick.

  Args:
      individual_player_cards: Dictionary where keys are players and values are lists of their remaining cards (excluding diamonds).
      card_to_win: Integer value of the card that needs to win.

  Returns:
      Float: Probability of the card winning the trick.
  """

  all_cards = reduce ( lambda x,y : x | y, [ player_cards for player_cards in individual_player_cards ] )

  favorable_cards = sum([card.get_value() < card_to_win.get_value() for card in all_cards])

  # Total possible combinations (product of remaining cards for each player)
  total_combinations = reduce( lambda x, y : x * y, [ len(player_cards) for player_cards in individual_player_cards ] )

  return favorable_cards / total_combinations

def probabilistic_bidding(hand, remaining_diamonds, other_players_cards):
  """
  This function chooses the card to bid based on expected points.

  Args:
      hand: List of card values (integers) in your hand.
      revealed_diamonds: List of revealed diamond values (integers).

  Returns:
      Integer: The card value to bid (from your hand).
  """
  expected_points = calculate_expected_points(hand, remaining_diamonds, other_players_cards)
  best_card = max(expected_points, key=expected_points.get)
  return best_card

def calculate_expected_points(hand, remaining_diamonds, other_player_cards):
  """
  This function calculates the expected points for bidding each card in your hand.

  Args:
      hand: List of card values (integers) in your hand.
      _: Placeholder argument for revealed_diamonds (not used).

  Returns:
      Dictionary: Expected points for bidding each card in your hand.
  """

  def get_expected_points_for_card ( card_in_hand, remaining_diamonds_prob ):

    expected_points_with_diamond_prob = sum( card.get_value() * prob for card, prob in remaining_diamonds_prob.items()) / len ( remaining_diamonds_prob )
    expected_points_with_winning_prob = expected_points_with_diamond_prob * probability_of_winning ( other_player_cards, card_in_hand )

    return expected_points_with_winning_prob

  remaining_diamonds_prob = calculate_diamond_probability ( remaining_diamonds )

  expected_points = { card_in_hand : get_expected_points_for_card ( card_in_hand, remaining_diamonds_prob ) for card_in_hand in hand}

  return expected_points

def probabilistic_bidding(hand, remaining_diamonds, other_players_cards):
  """
  This function chooses the card to bid based on expected points.

  Args:
      hand: List of card values (integers) in your hand.
      revealed_diamonds: List of revealed diamond values (integers).

  Returns:
      Integer: The card value to bid (from your hand).
  """
  expected_points = calculate_expected_points(hand, remaining_diamonds, other_players_cards)
  best_card = max(expected_points, key=expected_points.get)
  return best_card



In [50]:
class AI_player:
  def __init__(self, name = "AI Player" ):
    self.name = name
    self.deck = []  # Add deck attribute to store player's unique cards
    self.points = 0
    self.remaining_diamonds = []

    self.other_player_cards = dict()

  def draw_card(self):
    if len(self.deck) > 0:
        self.deck.pop()  # Draw from player's deck
    else:
      print(f"{self.name}'s deck is empty!")

  def observe(self, all_cards, other_players_cards, banker_cards ):

    def observe_other_players( all_cards : list , other_players_cards : dict ) -> None:
        for player, cards in other_players_cards.items():
            self.other_player_cards [ player ] = set(all_cards) - set(self.other_player_cards.get ( player, set() ) | set([cards]))

    observe_other_players( all_cards, other_players_cards )
    self.remaining_diamonds = banker_cards

  def bid(self):
    if self.other_player_cards is not None:
        chosen_card = probabilistic_bidding( self.deck , self.remaining_diamonds,
                                    [ cards for cards in self.other_player_cards.values()] )
        #print([ len(cards) for cards in self.other_player_cards.values()])
    else:
        chosen_card = random.choice(self.deck)

    #print(f"{self.name} bids: {chosen_card.value}")
    return chosen_card

  def update_points(self, round_points):
    self.points += round_points

In [98]:
all_cards = [Card(suit, value) for suit in ["Spades", "Hearts", "Clubs"] for value in card_values.keys()]
random.shuffle(all_cards)

def initialize_game(ai_player, num_players):
  # Create players, remove diamonds from deck, and distribute unique decks
  players = [Player(f"Player {i+1}") for i in range(num_players)]

  for index, player in enumerate(players + [ ai_player ]):
    #print(index)
    player.deck = all_cards [ index * 13 : (index + 1) * 13 ]

  return players

def get_card_name( card : Card ) -> str:
  return card.suit + "_" + str(card.value)

def play_round(players, banker, ai_player):
  # Bidding phase
  players_and_bids = { player.name : player.bid() for player in players }
  ai_player.observe( all_cards, players_and_bids, banker.diamond_deck )
  ai_card = ai_player.bid()

  # Reveal phase and calculate points
  revealed_diamond_card = banker.generate_diamond_card()
  round_winner_card_value = max( [ bidded_card.get_value() for bidded_card in players_and_bids.values() ] + [ai_card.get_value()])
  round_points = revealed_diamond_card.get_value()

  # Handle ties or multiple winners
  winning_players = [player for player in players if players_and_bids [ player.name ].get_value() == round_winner_card_value]
  if ai_card.get_value() == round_winner_card_value: winning_players.append(ai_player)

  if len(winning_players) > 1:
    round_points /= len(winning_players)  # Split points for ties

  # Update player points
  for player in winning_players:
    player.update_points(round_points)

  # Print round results
  player_and_cards = { player : get_card_name(players_and_bids[player]) for player in players_and_bids }
  player_and_cards.update({ ai_player.name : get_card_name(ai_card)})
  player_and_cards.update({ "Banker" : get_card_name(revealed_diamond_card) })
  
  winners = [player.name for player in winning_players]
 
  return player_and_cards, winners, str(round_points)


In [99]:
def play_game(num_players, num_rounds = 13):
  ai_player = AI_player()
  players = initialize_game(ai_player, num_players)
  banker = Banker()

  for round_num in range(1, num_rounds + 1):
    print(f"\n** Round {round_num} **")
    print(play_round(players, banker, ai_player ))

  # Print final scores
  print("\n** Final Scores **")
  for player in players:
    print(f"{player.name}: {player.points} points")

In [101]:
play_game( 2 , 3 )


** Round 1 **
({'Player 1': 'Hearts_ace', 'Player 2': 'Hearts_2', 'AI Player': 'Spades_ace', 'Banker': 'Diamonds_6'}, ['Player 1', 'AI Player'], '3.0')

** Round 2 **
({'Player 1': 'Clubs_3', 'Player 2': 'Spades_7', 'AI Player': 'Clubs_10', 'Banker': 'Diamonds_king'}, ['AI Player'], '13')

** Round 3 **
({'Player 1': 'Clubs_7', 'Player 2': 'Hearts_8', 'AI Player': 'Spades_ace', 'Banker': 'Diamonds_2'}, ['AI Player'], '2')

** Final Scores **
Player 1: 3.0 points
Player 2: 0 points


In [102]:
import pygame
import random
import time
import os

def play_game(num_players, num_rounds = 13):

  ai_player = AI_player()
  players = initialize_game(ai_player, num_players)
  banker = Banker()
  
  # Define screen size
  screen_width = 800
  screen_height = 600

  # Initialize pygame
  pygame.init()

  # Create the screen
  screen = pygame.display.set_mode((screen_width, screen_height))

  # Set window title
  pygame.display.set_caption("Four Player Bidding")

  # Define some colors
  background_color = (200, 200, 200)
  text_color = (0, 0, 0)
  yellow = (255, 255, 0)  # Color for the yellow box
  red = ( 255, 0, 0 )

  # Create a font for text
  font = pygame.font.Font(None, 32)

  # Define player areas as rectangles
  rectangle_width = 250
  rectangle_height = 100
  yellow_box_width = 100  # Width of the yellow box

  # Calculate positions for top and bottom rectangles
  top_left_x = 50  # Center horizontally
  top_left_y = 50  # Distance from top
  bottom_left_x = top_left_x
  bottom_left_y = screen_height - rectangle_height - 50  # Distance from bottom

  # Create rectangles for player areas and yellow boxes
  player_areas = []
  yellow_boxes = []
  gap = 20  # Gap between player areas

  # Load an image (replace "image.png" with your actual image path)
  image = pygame.image.load("image.png")  # Make sure the image path is correct
  image_width = 50  # Get the image width
  image_height = 70  # Get the image height
  image = pygame.transform.scale(image, (image_width, image_height))
  #print ( image_width, image_height )

  #List for card value and card's image 
  diamonds_card_val_image = {
    "Diamonds_" + str(index) : \
    pygame.image.load(os.path.join("cards", str(index) + "_of_diamonds.png")) \
    for index in range ( 2, 11 )
  }

  diamonds_card_val_image.update({
    "Diamonds_" + card: \
    pygame.image.load(os.path.join("cards", card + "_of_diamonds.png" )) \
    for card in ["ace","jack","king","queen"]
  })

  spades_card_val_image = {
    "Spades_" + str(index) : \
    pygame.image.load(os.path.join("cards", str(index) + "_of_spades.png")) \
    for index in range ( 2, 11 )
  }

  spades_card_val_image.update({
    "Spades_" + card: \
    pygame.image.load(os.path.join("cards", card + "_of_spades.png" )) \
    for card in ["ace","jack","king","queen"]
  })

  clubs_card_val_image = {
    "Clubs_" + str(index) : \
    pygame.image.load(os.path.join("cards", str(index) + "_of_clubs.png")) \
    for index in range ( 2, 11 )
  }

  clubs_card_val_image.update({
    "Clubs_" + card: \
    pygame.image.load(os.path.join("cards", card + "_of_clubs.png" )) \
    for card in ["ace","jack","king","queen"]
  })

  hearts_card_val_image = {
    "Hearts_" + str(index) : \
    pygame.image.load(os.path.join("cards", str(index) + "_of_hearts.png")) \
    for index in range ( 2, 11 )
  }

  hearts_card_val_image.update({
    "Hearts_" + card: \
    pygame.image.load(os.path.join("cards", card + "_of_hearts.png" )) \
    for card in ["ace","jack","king","queen"]
  })

  other_cards = {**diamonds_card_val_image, **hearts_card_val_image, 
                **clubs_card_val_image, **spades_card_val_image}

  for i in range(4):
    # Player area rectangle
    player_rect = pygame.Rect(top_left_x if i % 2 == 0 else top_left_x + rectangle_width + gap, 
                              top_left_y if i < 2 else bottom_left_y, 
                              rectangle_width, rectangle_height)
    player_areas.append(player_rect)

    # Yellow box rectangle
    yellow_box_rect = pygame.Rect(player_rect.right - yellow_box_width, player_rect.top, yellow_box_width, player_rect.height)
    yellow_boxes.append(yellow_box_rect)

  running = True
  for index in range(num_rounds):
    players_and_cards, winner, round_points  = play_round(players, banker, ai_player )
    print ( players_and_cards, "\n", winner )
    player_names = ["Banker"] + list(players_and_cards.keys())
    # Check for events (like closing the window)
    for event in pygame.event.get():
      if event.type == pygame.QUIT:
        running = False

    # Fill the screen with background color
    screen.fill(background_color)

    # Render images inside player areas (adjust position as needed)
    for i, area in enumerate(player_areas):
      # Center the image horizontally and vertically within the area
      image_x = area.centerx - image_width
      image_y = area.centery - image_height // 2
      screen.blit(image, (image_x, image_y))

    banker_box = yellow_boxes[0]
    image_x = banker_box.centerx - image_width // 2
    image_y = banker_box.centery - image_height // 2
    #print ( len(list(diamonds_card_val_image.values())) )
    yellow_box_image = list(diamonds_card_val_image.values())[random.randint(0,12)]
    yellow_box_image = pygame.transform.scale(yellow_box_image, 
                                              (image_width, image_height))
    screen.blit( yellow_box_image, ( image_x, image_y ) )

    # Draw yellow boxes with other players
    for i, box in enumerate(yellow_boxes[1:]):
      yellow_box_image = other_cards[players_and_cards[player_names[i]]]
      yellow_box_image = pygame.transform.scale(yellow_box_image, 
                                              (image_width, image_height))
      image_x = box.centerx - image_width // 2
      image_y = box.centery - image_height // 2
      screen.blit(yellow_box_image, (image_x, image_y))

    # Render and display player names
    for i, area in enumerate(player_areas):
      name_text = player_names[i]
      name_surface = font.render(name_text, True, text_color)
      # Center the name text horizontally below the area
      name_x = area.centerx - name_surface.get_width() // 2
      name_y = area.bottom + 10
      screen.blit(name_surface, (name_x, name_y))
    # Winner logic (replace with your actual logic to determine the winner)

    # Calculate winner text surface
    winner_text = "Winners : " + " ".join(winner) + " Points won : " + round_points
    winner_text_surface = font.render(winner_text, True, text_color)

    # Calculate winner text position
    screen_center_x = screen_width // 2
    screen_center_y = screen_height // 2
    winner_text_width = winner_text_surface.get_width()
    winner_text_height = winner_text_surface.get_height()
    winner_text_x = screen_center_x - winner_text_width // 2
    winner_text_y = (top_left_y + bottom_left_y) // 2 - winner_text_height // 2

    # Render winner text
    screen.blit(winner_text_surface, (winner_text_x, winner_text_y))

    # Update the display
    pygame.display.flip()
    time.sleep(2)  # Uncomment to add delay

  # Quit pygame
  pygame.quit()


In [103]:
play_game ( 2 , 3 )

{'Player 1': 'Hearts_6', 'Player 2': 'Spades_2', 'AI Player': 'Spades_ace', 'Banker': 'Diamonds_3'} 
 ['AI Player']
{'Player 1': 'Spades_8', 'Player 2': 'Spades_7', 'AI Player': 'Clubs_10', 'Banker': 'Diamonds_king'} 
 ['AI Player']
{'Player 1': 'Clubs_7', 'Player 2': 'Spades_9', 'AI Player': 'Spades_ace', 'Banker': 'Diamonds_10'} 
 ['AI Player']
